In [1]:
import chaospy as cp
import numpy as np
import pandas as pd
import respy as rp
from os import getpid
import time
from joblib import Parallel, delayed

## Produce input parameters

Draw the sample of random input parameters.   

In [12]:
# load model specifications
_, base_options = rp.get_example_model("kw_94_one", with_data=False)
base_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")
constraints = rp.get_parameter_constraints("kw_94_one")
# constraints.remove({"loc": "shocks_sdcorr", "type": "sdcorr"})
# mean and cov for sampling
mean = base_params["value"].to_numpy()[:27]
cov = pd.read_pickle("./input/respy-se-collection/covariance_kw_94_one.pkl").to_numpy()

# sample input parameters
np.random.seed(123)
distribution = cp.MvNormal(loc=mean, scale=cov)
input_params_20 = list(distribution.sample(20).T) # 20 draws for simplicity

In [36]:
def params_to_respy(input_params, *args):
    """transfer sampled paramters to respy format."""
    
    # baseline options and params for the indices.
    _, base_options = rp.get_example_model("kw_94_one", with_data=False)
    base_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")

    params_idx = pd.Series(data=input_params, index=base_params.index[0:27])

    assert len(params_idx) == 27, "Length of KW94 vector must be 27."
    part_1 = params_idx

    rp_params, _ = rp.get_example_model("kw_94_one", with_data=False)
    part_2 = rp_params.iloc[27:31, 0]

    parts = [part_1, part_2]
    rp_params_series = pd.concat(parts)
    input_params_respy = pd.DataFrame(rp_params_series, columns=["value"])

    return input_params_respy

In [52]:
start = time.time()
input_params_respy = Parallel(n_jobs=-1)(delayed(params_to_respy)(i) for i in input_params_20)
end = time.time()
# print(params_idx_respy)
print(f'\nTime to complete: {end - start:.2f}s\n')


Time to complete: 4.36s



In [54]:
input_params_respy[19]

value
category            name                                 
delta               delta                        0.950049
wage_a              constant                     9.210243
                    exp_edu                      0.037991
                    exp_a                        0.033133
                    exp_a_square                -0.000506
                    exp_b                        0.001611
                    exp_b_square                -0.000050
wage_b              constant                     8.479490
                    exp_edu                      0.069971
                    exp_b                        0.066694
                    exp_b_square                -0.000988
                    exp_a                        0.022431
                    exp_a_square                -0.000529
nonpec_edu          constant                   -95.996678
                    at_least_twelve_exp_edu    175.206036
                    not_edu_last_period      -4108.078119
nonpec_home         constant                 17714.796211
shocks_chol         chol_a                       0.198079
                    chol_b_a                     0.006159
                    chol_b                       0.251170
                    chol_edu_a                  36.722253
                    chol_edu_b                 -97.299220
                    chol_edu                  1624.481661
                    chol_home_a                -45.284234
                    chol_home_b                103.291629
                    chol_home_edu              -12.437761
                    chol_home                 1557.396521
lagged_choice_1_edu probability                  1.000000
initial_exp_edu_10  probability                  1.000000
maximum_exp         edu                         20.000000

## Simulation

In [55]:
# wrap simulation function
def simulation(input_params_respy):
    _, base_options = rp.get_example_model("kw_94_one", with_data=False)
    
    simulate = rp.get_simulate_func(input_params_respy, base_options)
    df = simulate(input_params_respy)
    
    print("I'm process", getpid())
    
    return df

### When I use the 1 CPU to compute, it works fine:

In [62]:
start = time.time()
df = Parallel(n_jobs=1)(delayed(simulation)(params) for params in input_params_respy)
end = time.time()
print(f'\nTime to complete: {end - start:.2f}s\n')

I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597
I'm process 69597

Time to complete: 118.82s



In [64]:
df[0].head(10)

Experience_A  Experience_B  Experience_Edu Lagged_Choice_1  \
Identifier Period                                                               
0          0                  0             0              10             edu   
           1                  1             0              10               a   
           2                  1             1              10               b   
           3                  1             2              10               b   
           4                  1             2              11             edu   
           5                  1             2              12             edu   
           6                  1             2              13             edu   
           7                  1             2              14             edu   
           8                  1             3              14               b   
           9                  1             4              14               b   

                   Shock_Reward_A  Meas_Error_Wage_A  Shock_Reward_B  \
Identifier Period                                                      
0          0            -0.035035                  1        0.040965   
           1            -0.354560                  1        1.185316   
           2            -1.063705                  1        1.245234   
           3            -0.692603                  1       -1.238533   
           4            -0.943424                  1       -0.581919   
           5             0.768313                  1       -1.989945   
           6             0.087829                  1       -0.598471   
           7             0.301492                  1        1.268182   
           8            -0.948111                  1        1.996244   
           9             0.796574                  1       -0.375418   

                   Meas_Error_Wage_B  Shock_Reward_Edu  Meas_Error_Wage_Edu  \
Identifier Period                                                             
0          0                       1         -0.713137                    1   
           1                       1          0.066755                    1   
           2                       1         -0.130574                    1   
           3                       1          0.768294                    1   
           4                       1          0.432645                    1   
           5                       1         -0.002513                    1   
           6                       1         -0.140483                    1   
           7                       1          0.982899                    1   
           8                       1         -0.370696                    1   
           9                       1          0.332443                    1   

                   ...  Nonpecuniary_Reward_Edu  Wage_Edu  Flow_Utility_Edu  \
Identifier Period  ...                                                        
0          0       ...              -155.639237       NaN      -1349.399531   
           1       ...             -4568.755850       NaN      -4502.698032   
           2       ...             -4568.755850       NaN      -4703.530244   
           3       ...             -4568.755850       NaN      -3027.752613   
           4       ...              -155.639237       NaN        807.175186   
           5       ...              -106.958418       NaN        -67.231687   
           6       ...              -106.958418       NaN       -301.978637   
           7       ...              -106.958418       NaN       1362.458191   
           8       ...             -4520.075031       NaN      -5152.111987   
           9       ...             -4520.075031       NaN      -4078.977756   

                   Value_Function_Edu Continuation_Value_Edu  \
Identifier Period                                              
0          0            357791.571408          377917.179533   
           1            355839.858910          379181.585519   
           2            361

### However, when I specify more than one CPU cores to perform parallel computing, it doesn't work:

In [69]:
# the number of CPUs on the local machine is 8
import joblib
num_cpu = joblib.cpu_count()

8

In [74]:
# with all processors
start = time.time()
simulate = Parallel(n_jobs=num_cpu)(delayed(get_simulate_func)(params) for params in input_params_respy)
end = time.time()
print(f'\nTime to complete: {end - start:.2f}s\n')

TypeError: cannot pickle '_nrt_python._MemInfo' object

In [73]:
# with 2 processors
start = time.time()
simulate = Parallel(n_jobs=2)(delayed(get_simulate_func)(params) for params in input_params_respy)
end = time.time()
print(f'\nTime to complete: {end - start:.2f}s\n')

FileNotFoundError: [Errno 2] No such file or directory: '8_1111.parquet'